In [1]:
!pip install opencv-python

In [2]:
import numpy as np
import cv2

In [3]:
def get_background(file_path):
    cap = cv2.VideoCapture(file_path)
    # we will randomly select 50 frames for the calculating the median
    frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    # we will store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames.append(frame)
    # calculate the median
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    return median_frame

In [4]:
INPUT_VIDEO_PATH = "input_files/Traffic_Laramie_2.mp4"
CONSECUTIVE_FRAMES = 2 # how manny frames will be compared to detect movement, less = more detailed detection, more computation needed
MINIMAL_SIZE = 1800 # minimal object size, big enough to detect cars but avoid pedestrians

In [5]:
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
# get the video frame height and width
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

save_name = f"output_files_task2/{INPUT_VIDEO_PATH.split('/')[-1]}"
# define codec and create VideoWriter object
out = cv2.VideoWriter(
    save_name,
    cv2.VideoWriter_fourcc(*'mp4v'), fps, 
    (frame_width, frame_height)
)

In [6]:
# get the background model
background = get_background(INPUT_VIDEO_PATH)
# convert the background model to grayscale format
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
frame_count = 0

In [7]:
import numpy as np
import time
car_count = 0
MAX_OBJECT_DISTANCE = 60
cords_set = set()
cords_to_delete_map = {}

def reset_counters():
    global car_count
    global cords_set
    car_count = 0
    cords_set = set()

def find_closest_cord(cord, cords_set):
    closest_dist = 100000000
    closest_cord = None
    for c in cords_set:
        point1 = np.array(cord)
        point2 = np.array(c)
        dist = np.linalg.norm(point1 - point2)
        if(dist < closest_dist):
            closest_dist = dist
            closest_cord = c 
    return closest_cord, closest_dist

def update_cord(current_cords, new_cors, cords_set):
    cords_set.remove(current_cords)
    cords_set.add(new_cors)

def draw_contours(contours, frame):
    global car_count
    global cords_set
    global cords_to_delete_map
    cords_in_frame = set()
    for contour in contours:
        # continue through the loop if contour area is less than MINIMAL_SIZE...
        # ... helps in removing noise detection
        if cv2.contourArea(contour) < MINIMAL_SIZE:
            continue
        # get the xmin, ymin, width, and height coordinates from the contours
        (x, y, w, h) = cv2.boundingRect(contour)
        X_THRESHOLD = 150
        if(y>280 and x<500 and ((x<X_THRESHOLD and y < 370) or (x>=X_THRESHOLD and y<400))):
            ### object tracking starts here
            cords_in_frame.add((x, y))
            closest_cord, distance = find_closest_cord((x, y), cords_set)
            if(distance <= MAX_OBJECT_DISTANCE):
                update_cord(closest_cord, (x,y), cords_set)
            else:
                car_count += 1
                cords_set.add((x, y))
            ### object tracking ends here
            # car is going to city center
            cv2.putText(frame, f"x:{str(x)}, y:{str(y)} dist: {str(int(distance))}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 
                       1, (0, 255, 0), 2, cv2.LINE_AA) 
            # draw the bounding boxes
            cv2.rectangle(frame, (x, y+2), (x+w, y+h), (0, 255, 0), 2)
    ### remove old cords from cords_set
    for cord in list(cords_set):
        closest_cord, distance = find_closest_cord(cord, cords_in_frame)
        if(distance > MAX_OBJECT_DISTANCE):
            key = f"{cord[0]}_{cord[1]}"
            if key not in cords_to_delete_map:
                 cords_to_delete_map[key] = 0
            cords_to_delete_map[key] +=1
            if(cords_to_delete_map[key] > 3):
                del cords_to_delete_map[key] 
                print("deleting cord from cords set(is out of frame): ", key)
                cords_set.remove(cord)
    ##### END old cord removal
    cv2.putText(frame, f"Car count: {str(car_count)} ", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
           1, (0, 255, 0), 2, cv2.LINE_AA) 

        

In [8]:
reset_counters()
frame_diff_list = []
frames_to_write = []
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame_count += 1
        orig_frame = frame.copy()
        # IMPORTANT STEP: convert the frame to grayscale first
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if frame_count % CONSECUTIVE_FRAMES == 0 or frame_count == 1:
            frame_diff_list = []
        # find the difference between current frame and base frame
        frame_diff = cv2.absdiff(gray, background)
        # thresholding to convert the frame to binary
        ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
        
        
        ### mask only street we care about

        # a mask is the same size as our image, but has only two pixel
        # values, 0 and 255 -- pixels with a value of 0 (background) are
        # ignored in the original image while mask pixels with a value of
        # 255 (foreground) are allowed to be kept
        mask = np.zeros(thres.shape[:2], dtype="uint8")
        mask2 = np.zeros(thres.shape[:2], dtype="uint8")
        mask3 = np.zeros(thres.shape[:2], dtype="uint8")
        cv2.rectangle(mask, (0, 200), (80, 400), 255, -1)
        cv2.rectangle(mask2, (80, 200), (250, 420), 255, -1)
        cv2.rectangle(mask3, (250, 200), (500, 450), 255, -1)
        mask4 = cv2.bitwise_or(cv2.bitwise_or(mask, mask2), mask3)

        
        # apply our mask -- notice how only the person in the image is
        # cropped out
        thres = cv2.bitwise_and(thres, thres, mask=mask4)
        
        
        ### end of masking
        
        
        # dilate the frame a bit to get some more white area...
        # ... makes the detection of contours a bit easier
        dilate_frame = cv2.dilate(thres, None, iterations=2)

        # append the final result into the `frame_diff_list`
        frame_diff_list.append(dilate_frame)
        # if we have reached `CONSECUTIVE_FRAMES` number of frames
        if len(frame_diff_list) == CONSECUTIVE_FRAMES:
            # add all the frames in the `frame_diff_list`
            sum_frames = sum(frame_diff_list)
            
            # find the contours around the white segmented areas
            contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            
            
            frames_to_write.append((orig_frame, contours))
            draw_contours(contours, frame)
            cv2.imshow('Detected Objects', frame)
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
        else:
            # write intermediate frames to avoid frame skipping in output video? 
            frames_to_write.append((orig_frame, []))
    else:
        break


deleting cord from cords set(is out of frame):  0_332
deleting cord from cords set(is out of frame):  0_350
deleting cord from cords set(is out of frame):  0_341
deleting cord from cords set(is out of frame):  0_341


In [9]:
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration_sec = frame_count/fps
number_of_minutes = duration_sec/60
cars_per_minute = car_count / number_of_minutes
print("cars per minute: ",cars_per_minute)
print("total car count: ", car_count)

cars per minute:  2.271006813020439
total car count:  4


In [ ]:
reset_counters()
i=0
for frame, contours in frames_to_write:
    j=1
    while(not contours and (i+j) <len(frames_to_write)):
        contours = frames_to_write[i+j][1]
        j+=1
    i+=1
    draw_contours(contours, frame)
    out.write(frame)
    
cap.release()
out.release()
cv2.destroyAllWindows()

deleting cord from cords set(is out of frame):  0_332
deleting cord from cords set(is out of frame):  0_350
deleting cord from cords set(is out of frame):  0_341
